In [1]:
import numpy as np
import pandas as pd
import altair as alt
import mercury as mr

## Sales in Q1, Q2 2025 📊

This dashboard gives a clear and simple overview of sales performance for the first two quarters of 2025. Use the filters on the left side to answer specific business questions. This dashboard was built in Python using the Mercury framework.



In [2]:
# Example sales DataFrame

rng = np.random.default_rng(7)
dates = pd.date_range("2025-01-01", "2025-06-30", freq="D")
regions = ["North", "South", "West", "East"]
channels = ["Online", "Retail", "Partners"]
products = ["Starter", "Pro", "Enterprise"]

n = 2500 # number of rows
df = pd.DataFrame(
    {
        "date": rng.choice(dates, size=n),
        "region": rng.choice(regions, size=n, p=[0.28, 0.22, 0.25, 0.25]),
        "channel": rng.choice(channels, size=n, p=[0.55, 0.35, 0.10]),
        "product": rng.choice(products, size=n, p=[0.55, 0.35, 0.10]),
        "units": rng.integers(1, 9, size=n),
    }
)

# Simple pricing model
price_map = {"Starter": 49, "Pro": 129, "Enterprise": 399}
df["unit_price"] = df["product"].map(price_map).astype(float)
df["revenue"] = (df["units"] * df["unit_price"]).round(2)

In [3]:
region_sel = mr.MultiSelect(label="Region", choices=regions, value=regions, position="inline")
granularity = mr.Select(label="Time granularity", choices=["Day", "Week", "Month"], value="Week", position="inline")

In [4]:
# filter data
mask = (df["region"].isin(region_sel.value))
dff = df.loc[mask].copy()

In [5]:
total_revenue = float(dff["revenue"].sum())
total_units = int(dff["units"].sum())
orders = int(len(dff))
aov = (total_revenue / orders) if orders else 0.0

In [6]:
mr.Markdown(
    f"""
### Report scope

- **Regions:** {region_sel.value}
- **Granularity:** {granularity.value}

### Key results

- **Total revenue:** {total_revenue:,.0f}
- **Units sold:** {total_units:,.0f}
- **Orders:** {orders:,.0f}
- **Average order value (AOV):** {aov:,.2f}
"""
)

MarkdownWidget(value="<h3>Report scope</h3>\n<ul>\n<li><strong>Regions:</strong> ['North', 'South', 'West', 'E…

The chart below aggregates the data by the selected time granularity and splits lines by region.


In [10]:
dff["date"] = pd.to_datetime(dff["date"])

if granularity.value == "Day":
    dff["period"] = dff["date"].dt.date.astype("datetime64[ns]")
elif granularity.value == "Week":
    dff["period"] = dff["date"].dt.to_period("W").dt.start_time
else:  # Month
    dff["period"] = dff["date"].dt.to_period("M").dt.start_time

agg = (
    dff.groupby(["period", "region"], as_index=False)
    .agg(revenue=("revenue", "sum"), units=("units", "sum"))
    .sort_values("period")
)

y_field = "revenue" 
y_title = "Revenue" 

chart = (
    alt.Chart(agg)
    .mark_line(point=True)
    .encode(
        x=alt.X("period:T", title=""),
        y=alt.Y(f"{y_field}:Q", title=y_title),
        color=alt.Color("region:N", title="Region"),
        tooltip=[
            alt.Tooltip("period:T", title="Period"),
            alt.Tooltip("region:N", title="Region"),
            alt.Tooltip("revenue:Q", title="Revenue", format=",.2f"),
            alt.Tooltip("units:Q", title="Units", format=",.0f"),
        ],
    )
    .properties(height=320, width=700)
    .interactive()
)

In [11]:
chart

alt.Chart(...)

## Conclusions

Sales are very good